## mannix/defog-llama3-sqlcoder-8b

### Hardware Specification --> RTX 3090 24GB

In [1]:
# !pip install ollama
# !pip install sqlparse
# !pip install langchain-core
# !pip install langchain_community

In [4]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import sqlparse
import time

In [5]:
template = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate a SQL query to answer this question: `{question}`

DDL statements:

CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,               -- Unique identifier for the customer
    FirstName VARCHAR(50),                    -- Customer's first name
    LastName VARCHAR(50),                     -- Customer's last name
    Email VARCHAR(100),                       -- Customer's email address
    Phone VARCHAR(15),                        -- Customer's phone number
    Address VARCHAR(200)                      -- Customer's address
);


CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,                  -- Unique identifier for the order
    CustomerID INT,                           -- ID of the customer who placed the order
    OrderDate DATE,                           -- Date the order was placed
    ShipperID INT,                            -- ID of the shipper handling the order
    TotalAmount DECIMAL(10, 2),               -- Total amount for the order
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),  -- Foreign key to Customers table
    FOREIGN KEY (ShipperID) REFERENCES Shippers(ShipperID)      -- Foreign key to Shippers table
);


CREATE TABLE Products (
    ProductID INT PRIMARY KEY,                -- Unique identifier for the product
    ProductName VARCHAR(100),                 -- Name of the product
    SupplierID INT,                           -- ID of the supplier providing the product
    CategoryID INT,                           -- ID of the category the product belongs to
    UnitPrice DECIMAL(10, 2),                 -- Price per unit of the product
    UnitsInStock INT,                         -- Number of units currently in stock
    FOREIGN KEY (SupplierID) REFERENCES Suppliers(SupplierID),  -- Foreign key to Suppliers table
    FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)  -- Foreign key to Categories table
);

CREATE TABLE Categories (
    CategoryID INT PRIMARY KEY,               -- Unique identifier for the category
    CategoryName VARCHAR(50)                  -- Name of the category
);

CREATE TABLE OrderDetails (
    OrderDetailID INT PRIMARY KEY,            -- Unique identifier for the order detail
    OrderID INT,                              -- ID of the order
    ProductID INT,                            -- ID of the product
    Quantity INT,                             -- Quantity of the product ordered
    UnitPrice DECIMAL(10, 2),                 -- Unit price of the product at the time of the order
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),          -- Foreign key to Orders table
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)     -- Foreign key to Products table
);

CREATE TABLE Suppliers (
    SupplierID INT PRIMARY KEY,               -- Unique identifier for the supplier
    SupplierName VARCHAR(100),                -- Name of the supplier
    ContactName VARCHAR(50),                  -- Contact person's name at the supplier
    ContactEmail VARCHAR(100),                -- Contact person's email at the supplier
    Phone VARCHAR(15),                        -- Supplier's phone number
    Address VARCHAR(200)                      -- Supplier's address
);

CREATE TABLE Shippers (
    ShipperID INT PRIMARY KEY,                -- Unique identifier for the shipper
    ShipperName VARCHAR(100),                 -- Name of the shipper
    Phone VARCHAR(15)                         -- Shipper's phone number
);

CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,               -- Unique identifier for the employee
    FirstName VARCHAR(50),                    -- Employee's first name
    LastName VARCHAR(50),                     -- Employee's last name
    Email VARCHAR(100),                       -- Employee's email address
    Phone VARCHAR(15),                        -- Employee's phone number
    HireDate DATE                             -- Date the employee was hired
);

CREATE TABLE Reviews (
    ReviewID INT PRIMARY KEY,                 -- Unique identifier for the review
    ProductID INT,                            -- ID of the product being reviewed
    CustomerID INT,                           -- ID of the customer who wrote the review
    Rating INT,                               -- Rating given by the customer (1-5)
    Comment VARCHAR(500),                     -- Review comment
    ReviewDate DATE,                          -- Date the review was written
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID),    -- Foreign key to Products table
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)  -- Foreign key to Customers table
);

CREATE TABLE Inventory (
    InventoryID INT PRIMARY KEY,              -- Unique identifier for the inventory record
    ProductID INT,                            -- ID of the product in the inventory
    Quantity INT,                             -- Quantity of the product available in inventory
    Location VARCHAR(100),                    -- Location of the inventory
    LastUpdated DATE,                         -- Date when the inventory was last updated
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)    -- Foreign key to Products table
);

The following SQL query best answers the question `{question}`:
```sql
"""


In [6]:
llm = Ollama(base_url="http://172.27.222.2:11434",
    model="mannix/defog-llama3-sqlcoder-8b" , temperature = 0 
) 

In [7]:
prompt = PromptTemplate.from_template(template=template)
chain = prompt | llm | StrOutputParser()

### Q1- Ground Truth
SELECT FirstName

FROM Customers

WHERE CustomerID = 1;

In [6]:
t1 = time.perf_counter()
query = "What is the first name of the customer with CustomerID 1?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)   # Correct

SELECT c.FirstName FROM Customers c WHERE c.CustomerID = 1;
Time taken to run: 3.475586572999873


### Q2 - Ground Truth
SELECT ShipperName FROM Shippers WHERE ShipperID = 2;

In [8]:
t1 = time.perf_counter()
query = "What is the name of the shipper with ShipperID 2?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT s.ShipperName FROM Shippers s WHERE s.ShipperID = 2;
Time taken to run: 1.0060667090001516


### Q3-Ground Truth
SELECT UnitPrice FROM Products WHERE ProductID = 3;

In [9]:
t1 = time.perf_counter()
query = "What is the unit price of the product with ProductID 3?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # correct

SELECT p.UnitPrice FROM Products p WHERE p.ProductID = 3;
Time taken to run: 0.7808351650000986


### Q4-Ground Truth
SELECT c.FirstName, c.LastName

FROM Customers c

JOIN Orders o ON c.CustomerID = o.CustomerID

WHERE o.OrderID = 2;

In [10]:
t1 = time.perf_counter()
query = "Which customer placed the order with OrderID 2?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT c.FirstName, c.LastName FROM Customers c JOIN Orders o ON c.CustomerID = o.CustomerID WHERE o.OrderID = 2;
Time taken to run: 0.9637540409999019


### Q5-Ground Truth
SELECT TotalAmount FROM Orders

WHERE OrderDate = '2023-05-01';

In [8]:
t1 = time.perf_counter()
query = "What is the total amount of the order placed on '2023-05-01'?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)    # Wrong

# Explanation
'''These queries are not functionally the same:

Ground Truth: Retrieves the TotalAmount of the order(s) placed on '2023-05-01'.
Model: Retrieves the sum of TotalAmount for all orders placed on '2023-05-01'.'''

SELECT SUM(o.TotalAmount) AS TotalOrderAmount FROM Orders o WHERE o.OrderDate = '2023-05-01';
Time taken to run: 2.692882389999795


"These queries are not functionally the same:\n\nGround Truth: Retrieves the TotalAmount of the order(s) placed on '2023-05-01'.\nModel: Retrieves the sum of TotalAmount for all orders placed on '2023-05-01'."

### Q6-Ground Truth
SELECT UnitsInStock FROM Products WHERE ProductID = 5;

In [12]:
t1 = time.perf_counter()
query = "How many units of the product with ProductID 5 are currently in stock?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)   # Correct

SELECT p.UnitsInStock FROM Products p WHERE p.ProductID = 5;
Time taken to run: 0.8035030999999435


### Q7 - Ground Truth
SELECT SUM(od.Quantity) AS TotalQuantity FROM OrderDetails od JOIN Products p ON od.ProductID = p.ProductID WHERE p.ProductName = 'Smartphone';



In [10]:
t1 = time.perf_counter()
query = "What is the total quantity of 'Smartphone' ordered?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Wrong

#Explanation 

'''Ground Truth: Matches 'Smartphone' directly.
Model: Uses LOWER() function to make the comparison case-insensitive.
These queries are not functionally identical due to the case sensitivity handling.'''

SELECT SUM(od.Quantity) AS TotalQuantity FROM OrderDetails od JOIN Products p ON od.ProductID = p.ProductID WHERE LOWER(p.ProductName) = 'smartphone';
Time taken to run: 2.7588323010004387


"Ground Truth: Matches 'Smartphone' directly.\nModel: Uses LOWER() function to make the comparison case-insensitive.\nThese queries are not functionally identical due to the case sensitivity handling."

### Q8 - Ground Truth
SELECT c.FirstName, c.LastName FROM Customers c JOIN Reviews r ON c.CustomerID = r.CustomerID WHERE r.Rating = 5;

In [9]:
t1 = time.perf_counter()
query = "List the names of customers who have written reviews with a rating of 5.?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # correct

SELECT c.FirstName, c.LastName FROM Customers c JOIN Reviews r ON c.CustomerID = r.CustomerID WHERE r.Rating = 5;
Time taken to run: 0.933211248000589


"Ground Truth: Matches 'Smartphone' directly.\nModel: Uses LOWER() function to make the comparison case-insensitive.\nThese queries are not functionally identical due to the case sensitivity handling."

### Q9 - Ground Truth
SELECT AVG(r.Rating) AS AverageRating FROM Reviews r JOIN Products p ON r.ProductID = p.ProductID WHERE p.ProductName = 'Tablet';

In [11]:
t1 = time.perf_counter()
query = "What is the average rating of the product 'Tablet'?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Wrong

#Explanation
'''
Ground Truth: Matches 'Tablet' directly.
Model: Uses LOWER() function to make the comparison case-insensitive.

These queries are not functionally identical due to the case sensitivity handling.
'''

SELECT AVG(r.Rating) AS AverageRating FROM Reviews r JOIN Products p ON r.ProductID = p.ProductID WHERE LOWER(p.ProductName) = 'tablet';
Time taken to run: 0.9181779380014632


"\nGround Truth: Matches 'Tablet' directly.\nModel: Uses LOWER() function to make the comparison case-insensitive.\n\nThese queries are not functionally identical due to the case sensitivity handling.\n"

### Q10 - Ground Truth
SELECT SUM(o.TotalAmount) AS TotalSpent

FROM Orders o

JOIN Customers c ON o.CustomerID = c.CustomerID

WHERE c.FirstName = 'Alice' AND c.LastName = 'Johnson';



In [12]:
t1 = time.perf_counter()
query = "Find the total amount spent by the customer 'Alice Johnson'."
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)   # Wrong

#Explanation

'''
Ground Truth: Matches 'Alice' and 'Johnson' directly.
Model: Uses LOWER() function to make the comparison case-insensitive.

These queries are not functionally identical due to the case sensitivity handling.
'''

SELECT SUM(o.TotalAmount) AS TotalSpent FROM Orders o JOIN Customers c ON o.CustomerID = c.CustomerID WHERE LOWER(c.FirstName) = 'alice' AND LOWER(c.LastName) = 'johnson';
Time taken to run: 1.03976578000038


"\nGround Truth: Matches 'Alice' and 'Johnson' directly.\nModel: Uses LOWER() function to make the comparison case-insensitive.\n\nThese queries are not functionally identical due to the case sensitivity handling.\n"

### Q11 - Ground Truth
SELECT p.ProductName

FROM Products p

LEFT JOIN Reviews r ON p.ProductID = r.ProductID

WHERE r.ProductID IS NULL;

In [17]:
t1 = time.perf_counter()
query = "Which products have never been reviewed?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # correct

SELECT p.ProductName FROM Products p LEFT JOIN Reviews r ON p.ProductID = r.ProductID WHERE r.ProductID IS NULL;
Time taken to run: 0.8982297429997743


### Q12 - Ground Truth
SELECT FirstName, LastName

FROM Employees

ORDER BY HireDate

LIMIT 1;

In [18]:
t1 = time.perf_counter()
query = 'Which employee has been with the company the longest?'
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT e.FirstName, e.LastName FROM Employees e ORDER BY e.HireDate ASC LIMIT 1;
Time taken to run: 0.8303556199998638


### Q13 - Ground Truth
SELECT p.ProductName

FROM Products p

JOIN Suppliers s ON p.SupplierID = s.SupplierID

WHERE s.SupplierName = 'TechCorp';

In [19]:
t1 = time.perf_counter()
query ="List the names of all products provided by 'TechCorp'."
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT p.ProductName FROM Products p JOIN Suppliers s ON p.SupplierID = s.SupplierID WHERE LOWER(s.SupplierName) = 'techcorp';
Time taken to run: 0.9713725749998048


# <font color='orange'>Longer Queries below</font>

### Q14 - Ground Truth
SELECT c.FirstName, c.LastName

FROM Customers c

JOIN Orders o ON c.CustomerID = o.CustomerID

JOIN OrderDetails od ON o.OrderID = od.OrderID

JOIN Products p ON od.ProductID = p.ProductID

JOIN Categories cat ON p.CategoryID = cat.CategoryID

WHERE cat.CategoryName = 'Electronics'

GROUP BY c.CustomerID

HAVING COUNT(DISTINCT p.ProductID) = (SELECT COUNT(*) FROM Products

WHERE CategoryID = (SELECT CategoryID FROM Categories WHERE

CategoryName = 'Electronics'));

In [20]:
t1 = time.perf_counter()
query ="Find the names of customers who have ordered all products from the 'Electronics"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Wrong

SELECT c.FirstName, c.LastName FROM Customers c JOIN Orders o ON c.CustomerID = o.CustomerID JOIN OrderDetails od ON o.OrderID = od.OrderID JOIN Products p ON od.ProductID = p.ProductID WHERE p.CategoryName = 'Electronics' GROUP BY c.FirstName, c.LastName HAVING COUNT(DISTINCT od.ProductID) = (SELECT COUNT(*) FROM Products pr WHERE pr.CategoryName = 'Electronics') ORDER BY c.FirstName NULLS LAST;
Time taken to run: 1.611955129999842


### Q15 - Ground Truth
SELECT SUM(od.Quantity * od.UnitPrice) AS TotalRevenue

FROM OrderDetails od

JOIN Products p ON od.ProductID = p.ProductID

JOIN Suppliers s ON p.SupplierID = s.SupplierID

WHERE s.SupplierName = 'Gadgets Inc';



In [21]:
t1 = time.perf_counter()
query ="What is the total revenue generated from products supplied by 'Gadgets Inc'?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)   # Wrong

SELECT SUM(p.UnitPrice * od.Quantity) AS TotalRevenue FROM Products p JOIN OrderDetails od ON p.ProductID = od.ProductID WHERE LOWER(p.SupplierName) = 'gadgets inc';
Time taken to run: 1.0557892530000572


### Q16 - Ground Truth
SELECT p.ProductName

FROM Products p

LEFT JOIN OrderDetails od ON p.ProductID = od.ProductID

WHERE od.ProductID IS NULL;



In [22]:
t1 = time.perf_counter()
query ="List the names of products that have never been ordered."
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT p.ProductName FROM Products p LEFT JOIN OrderDetails od ON p.ProductID = od.ProductID WHERE od.ProductID IS NULL;
Time taken to run: 0.9000497460001498


### Q17 - Ground Truth
SELECT s.ShipperName, SUM(o.TotalAmount) AS TotalValue

FROM Shippers s

JOIN Orders o ON s.ShipperID = o.ShipperID

GROUP BY s.ShipperID

ORDER BY TotalValue DESC

LIMIT 1;

In [23]:
t1 = time.perf_counter()
query ="Which shipper has handled the highest total value of orders?"
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

WITH TotalOrderValue AS (SELECT o.ShipperID, SUM(o.TotalAmount) AS TotalShipperAmount FROM Orders o GROUP BY o.ShipperID) SELECT t.ShipperID, t.TotalShipperAmount FROM TotalOrderValue t ORDER BY t.TotalShipperAmount DESC NULLS LAST LIMIT 1;
Time taken to run: 1.2833004340000116


### Q18 - Ground Truth
SELECT DISTINCT c.FirstName, c.LastName FROM Customers c

JOIN Orders o ON c.CustomerID = o.CustomerID

JOIN OrderDetails od ON o.OrderID = od.OrderID

JOIN Products p ON od.ProductID = p.ProductID

WHERE o.ShipperID = (SELECT ShipperID FROM Shippers WHERE ShipperName = 'FastShip')

GROUP BY o.OrderID, c.CustomerID

HAVING COUNT(DISTINCT p.SupplierID) > 1;



In [24]:
t1 = time.perf_counter()
query ="Find the names of customers who have placed orders that were shipped by 'FastShip' and contain products from multiple suppliers."
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT c.FirstName, c.LastName FROM Customers c JOIN Orders o ON c.CustomerID = o.CustomerID WHERE o.ShipperID IN (SELECT ShipperID FROM Shippers WHERE ShipperName = 'FastShip') AND c.CustomerID IN (SELECT DISTINCT OrderID FROM OrderDetails od JOIN Products p ON od.ProductID = p.ProductID GROUP BY od.OrderID HAVING COUNT(DISTINCT p.SupplierID) > 1);
Time taken to run: 1.5666713589998835


### Q19 - Ground Truth
SELECT p.ProductName, SUM(od.Quantity * od.UnitPrice) AS TotalRevenue

FROM Products p

JOIN OrderDetails od ON p.ProductID = od.ProductID

GROUP BY p.ProductID

ORDER BY TotalRevenue DESC

LIMIT 1;

In [25]:
t1 = time.perf_counter()
query ='Identify the product with the highest total sales revenue.'
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)   # Correct

WITH ProductSales AS (SELECT p.ProductID, SUM(od.Quantity * od.UnitPrice) AS TotalRevenue FROM OrderDetails od JOIN Products p ON od.ProductID = p.ProductID GROUP BY p.ProductID) SELECT ps.ProductID, ps.TotalRevenue FROM ProductSales ps ORDER BY ps.TotalRevenue DESC NULLS LAST LIMIT 1;
Time taken to run: 1.316469228999722


### Q20 - Ground Truth
SELECT p.ProductName

FROM Products p

JOIN Reviews r ON p.ProductID = r.ProductID

GROUP BY p.ProductID

HAVING COUNT(r.ReviewID) > 1;

In [26]:
t1 = time.perf_counter()
query ='List the products that have been reviewed by more than one customer?'
response = chain.invoke(query)
print(sqlparse.format(response))
t2 = time.perf_counter()
print("Time taken to run:",t2-t1)  # Correct

SELECT p.ProductName FROM Reviews r JOIN Products p ON r.ProductID = p.ProductID GROUP BY p.ProductName HAVING COUNT(DISTINCT r.CustomerID) > 1 ORDER BY p.ProductName NULLS LAST;
Time taken to run: 1.104371194000123
